# Generating participant hierarchies

This notebook provides the tools for organizing the participants of a chapter into a semantic hierarchy based on synonymous and part-whole relationships. This is the first step for analyzing the participants of a text. Another notebook provides measures for collocating and disambiguating participants across a span of chapters (cf. "1b_Disambiguating and Collocating Participants.ipynb").

In [1]:
import os, sys, re
from collections import defaultdict
from anytree.exporter import DotExporter
from anytree import Node, RenderTree, findall, findall_by_attr, find_by_attr
import graphviz

In [2]:
#Specifying path to graphviz
os.environ["PATH"] += os.pathsep + 'C:/Users/Ejer/Anaconda3/Library/bin/graphviz'

In [3]:
from tf.app import use
A = use('etcbc/bhsa', version='c', hoist=globals(), mod='ch-jensen/participants/actor/tf')

**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10938.05,100
chapter,929,459.19,100
lex,9233,46.20,100
verse,23213,18.38,100
half_verse,45180,9.44,100
sentence,63727,6.69,100
sentence_atom,64525,6.61,100
clause,88121,4.84,100
clause_atom,90688,4.70,100
phrase,253207,1.68,100


In [4]:
import Nodes

**Locating corpus resources ...**

Ancillary functions:

In [5]:
def show(actor,allrefs,n=int()):
    '''
    This function shows all references pertaining to a particular participant label
    '''
    if not n:
        node = allrefs[actor][0]
    else:
        node = n
    A.pretty(L.u(node, 'verse')[0], highlights = {node: 'red'})
    
def checkList(allrefs):
    '''
    Simple ancillary function to check whether an actor has only one reference which is either a Pred, PreC, or PtcO.
    This function is useful because single-case actors are not normally included in the participant data.
    '''
    
    for actor in allrefs:
        if len(allrefs[actor]) == 1:
            ph = L.u(allrefs[actor][0], 'phrase')[0]
            func = F.function.v(ph)
            if func in {'Pred','PreC','PtcO'}:
                print(actor)

Functions for creating trees:

In [6]:
def edgetypefunc(node, child):
    return '--'

def createPNG(tree, book, chapter):
    title = f'trees/ParticipantTreeStructure_{book[:3]}_{chapter}.png'
    DotExporter(tree, graph="digraph").to_picture(title)
    
def createPNG_extract(tree, book, chapter, node, max_indentation):
    title = f'trees/ParticipantTreeStructure_subtree_{book[:3]}_{chapter}.png'
    subtree = find_by_attr(tree, node, maxlevel=max_indentation)
    DotExporter(subtree, graph="digraph").to_picture(title)    

## Lev 17

#### a. Listing all participants and their references

In [7]:
Lev17 = Nodes.GenerateNodes('Leviticus',17)
#GenerateNodes('Leviticus',17).actorDict()
Lev17.allRefs()
Lev17.AllRefs

{'JHWH': [943175,
  943176,
  943178,
  943187,
  943188,
  943189,
  943207,
  1317274,
  943228,
  943234,
  1317293,
  943246,
  943279,
  943295,
  63229,
  943302,
  943310,
  943311,
  943322,
  943360],
 'MCH=': [943177, 943179, 943182, 943264],
 '>HRN BN ->HRN KL BN JFR>L': [943180, 63028],
 '>HRN': [1317252, 63020],
 'BN JFR>L': [1317258,
  943219,
  943220,
  63100,
  943223,
  943224,
  943227,
  943232,
  943249,
  63153,
  943254,
  943255,
  63165,
  63167,
  63169,
  63255,
  943314,
  63262,
  943323,
  63279,
  63289,
  1317334,
  63306,
  943361,
  943364],
 'JFR>L': [1317257,
  1317263,
  1317283,
  1317305,
  1317317,
  1317327,
  1317333,
  1317345],
 'DBR': [943184, 943185],
 '>JC >JC': [943190,
  943193,
  943198,
  943204,
  943205,
  943211,
  943213,
  943215,
  943216,
  63093],
 '>JC': [1317261, 943285, 943286, 1317331],
 'BJT JFR>L': [943191, 1317306, 63183, 1317318, 63222],
 'CWR KFB <Z==': [943194, 63066],
 'MXNH': [943195, 943200],
 'PTX >HL MW<D': [9432

* '>JC >JC refers to a person belonging to the house of Israel
* '>JC >JC#2 refers to a person belonging to the house of Israel or a sojourner.

#### Identifying participants to include (here human/divine participants)

In [ ]:
humanRefs_Lev17 = ['JHWH', 'MCH=', '>HRN BN ->HRN KL BN JFR>L', '>HRN', 'BN JFR>L', '>JC >JC', '>JC', 'BJT JFR>L', 'KHN',
                   '>JC >JC#2', 'BJT JFR>L GR', 'GR', 'NPC', 'NPC#2', 'KL', 'MN <M/ ->JC >JC','MN QRB/ <M/ -NPC',
                   'MN QRB/ <M/ ->JC >JC','KL/ NPC/','F<JR=','B TWK/ -BJT JFR>L','B TWK/ -BJT JFR>L#2']

#### Organizing the participants into synonyms or hyponyms (part-whole relations)

This task is manual and obviously somewhat subjective.

In [ ]:
synonyms_Lev17 = {'>JC >JC': ['>JC'], 'KL/ NPC/': ['NPC#2'], '>JC >JC#2': ['NPC', 'KL'],
                  'BJT JFR>L':['B TWK/ -BJT JFR>L','B TWK/ -BJT JFR>L#2'], 'MN <M/ ->JC >JC':['MN QRB/ <M/ -NPC',
                                                                                              'MN QRB/ <M/ ->JC >JC']}

hyponyms_Lev17 = {'>HRN BN ->HRN KL BN JFR>L': ['>HRN', 'BN JFR>L'],'BJT JFR>L GR': ['BJT JFR>L','GR'],
                  '>JC >JC#2':['>JC >JC','GR']}

In [ ]:
#show('B TWK/ -BJT JFR>L',Lev17.AllRefs,943230)

A tree is created based on the synonyms and hyponyms defined:

In [ ]:
tree = Lev17.tree(humanRefs_Lev17, synonyms_Lev17, hyponyms_Lev17)

In [ ]:
Lev17.treeStructure()

The tree can be exported:

In [ ]:
createPNG(tree, 'Leviticus', 17)

A subset of the tree can be exported:

In [ ]:
#createPNG_extract(tree, 'Leviticus', 17, node='>JC >JC#2', max_indentation=2)

Finally, the resulting list of participants and their references can be generated:

In [ ]:
Lev17.resultRefs()

Compare resulting list with initial list of all references:

In [ ]:
error_list_Lev17 = Lev17.checkResults()
print(len(error_list_Lev17))

## Lev 18

In [ ]:
Lev18 = Nodes.GenerateNodes('Leviticus',18)

Lev18.allRefs()
Lev18.AllRefs

In [ ]:
humanRefs_Lev18 = ['JHWH','MCH=','BN JFR>L','>DM','>JC >JC','>B -2ms','>M -2ms','2ms','<RWH >CH >B -2ms','>CH >B -2ms',
                  'BT >B -2msBT >M -2ms','BN -2ms','C>R >B -2ms','C>R >M -2ms','>X >B -2ms','DWDH -2ms',
                   '>CH BN -2ms','<RWH >CH >X -2ms','>X -2ms','>CH','C>RH','>XWT ->CH','>CH <MJT -2ms','GWJ','GR','>JC >RY',
                   'KL','NPC','>L KL/ C>R/ BFR/ ->JC >JC','<RWH/ >B/ -2msW <RWH/ >M/ -2ms','<RWH/ ->M -2ms','<RWH/ -',
                   '<RWH/ -<RWH -2ms','<RWH/ -#2','<RWH/ >XWT/ >B/ -2ms','<RWH/ >XWT/ >M/ -2ms','<RWH/ >X/ >B/ -2ms',
                   '<RWH/ KLH/ -2ms','<RWH/ ->CH BN -2ms','<RWH/ >CH/ W BT/ ->CH','>T BT/ BN/ ->CH W >T BT/ BT/ ->CH',
                   '<RWH/ ->CH','<RWH/ ->CH#2','<RWH/ ->CH#3','MN ZR</ -2ms','L H MLK=/', '>T CM/ >LHJM/ -2ms','>T== ZKR=/',
                   '>T JCB[ ->RY','MN QRB/ <M/ -NPC','L PNH/ -BN JFR>L','MN PNH/ -BN JFR>L','B TWK/ -BN JFR>L']

landRefs_Lev18 = ['>RY MYRJM','>RY KN<N','>RY']

In [ ]:
synonyms_Lev18 = {'JHWH': ['>T CM/ >LHJM/ -2ms'],
                  'BN JFR>L': ['>JC >JC','L PNH/ -BN JFR>L','MN PNH/ -BN JFR>L','B TWK/ -BN JFR>L'],
                  '>JC >RY':['GWJ','>T JCB[ ->RY'], 'NPC': ['KL'], '>M -2ms':['<RWH/ ->M -2ms'],
                  'BT >B -2msBT >M -2ms':['<RWH/ -', '<RWH/ -#2'], '<RWH/ >XWT/ >B/ -2ms': ['C>R >B -2ms'],
                  '>CH >B -2ms':['<RWH >CH >B -2ms'],'<RWH/ >XWT/ >M/ -2ms': ['C>R >M -2ms'],
                 'DWDH -2ms': ['<RWH/ >X/ >B/ -2ms'], '>CH BN -2ms': ['<RWH/ KLH/ -2ms', '<RWH/ ->CH BN -2ms'],
                  '>CH': ['<RWH/ ->CH', '<RWH/ ->CH#3','<RWH/ ->CH#2']}

hyponyms_Lev18 = {'>L KL/ C>R/ BFR/ ->JC >JC': ['<RWH/ >B/ -2msW <RWH/ >M/ -2ms','>CH >B -2ms',
                                                                    'BT >B -2msBT >M -2ms','<RWH/ -<RWH -2ms',
                  '<RWH/ >XWT/ >B/ -2ms','<RWH/ >XWT/ >M/ -2ms','DWDH -2ms','>CH BN -2ms','<RWH >CH >X -2ms'],
                  '<RWH/ >CH/ W BT/ ->CH': ['>CH'], 'C>RH': ['<RWH/ >CH/ W BT/ ->CH','>T BT/ BN/ ->CH W >T BT/ BT/ ->CH'],
                  '<RWH/ >B/ -2msW <RWH/ >M/ -2ms': ['>B -2ms','>M -2ms']}

In [ ]:
show('>JC',Lev18.AllRefs)

#### Notes:
* '<RWH' refers to "nakedness" but also to the body of a person, therefore included as referring to the person.
* '>DM: Generic, 3ms. Relates to the addressees but as a general principle for how human beings should live (v. 5). This generic term for all human beings is interpreted as synonymous to KL and NPC (v. 29).
* '>JC >JC: Another generic term but clearly means "anyone of you", that is, a member of the addresses (synonym) (cf. 18:6).
* A change between plural and singular referenc in 18:7. Here, the two references are differentiated.
* KL: Is KL synonymous to "you" (pl.)?
* '>L KL/ C>R/ BFR/ ->JC >JC: Is a collective for all female relatives in the subsequent text (v. 6).
* 'MN ZR</ -2ms': Refers to both "your daughter" and "your son", but not the other way round.
* C>RH: Is a hypernym of >CH and her grandchildren (v. 17).
* '>CH' refers to a woman to which the person is not related. This reference (along with references to the woman's dauther and reference to the wife a fellow) is therefore not a logical hyponym of the reference in v. 6.

In [ ]:
tree = Lev18.tree(humanRefs_Lev18, synonyms_Lev18, hyponyms_Lev18)

In [ ]:
Lev18.treeStructure()
createPNG(tree, 'Leviticus', 18)

In [ ]:
Lev18.resultRefs()

In [ ]:
error_list_Lev18 = Lev18.checkResults()
print(len(error_list_Lev18))

## Lev 19

In [ ]:
Lev19 = Nodes.GenerateNodes('Leviticus',19)
Lev19.allRefs()
Lev19.AllRefs

#### Notes:

* Human references governed by L PNH ("in front of") are included, because the construction functions like a preposition.
* The soul (NPC) in v. 28 is dubious. Most likely, it does not denote a soul in general, cf. the other uses of NPC in the chapter.
* GR refers to BN JFR>L in this chapter, but since GR is the first full phrase referring to that referent, it is used as the label.
* MWT[ refers to the man and the slave girl having sexual intercourse
* SLX[ refers to the man who is being forgiven (a passive)

In [ ]:
humanRefs_Lev19 = ['JHWH','MCH=','GR','>JC','3ms','NPC','2ms','GR#2','>LHJM -2ms','R<= -2ms','<MJT -2ms','CPXH','KHN',
                  'BT -2ms','JD<NJ','MYRJM','>L KL/ <DH/ BN/ JFR>L/','>M/ ->JC W >B/ ->JC','>L H >LJL/','MN <M/ -NPC',
                   'L H <NJ/ W L H GR/', 'B CM/ -JHWH','>T CM/ >LHJM/ -2ms','XRC=/','PNH/ DL/','PNH/ GDWL/',
                   'B <MJT/ ->JC','L PNH/ <WR/','B <M/ -2ms','>T >X/ -2ms','>T BN/ <M/ -2ms','L >JC/',
                   'MWT[','SLX[','>L H >WB/','MN PNH/ FJBH/','PNH/ ZQN/']

land_references_Lev19 = ['>RY -GR','>RY','>RY MYRJM']

In [ ]:
synonyms_Lev19 = {'JHWH': ['>LHJM -2ms','B CM/ -JHWH','>T CM/ >LHJM/ -2ms',], 'GR': ['>L KL/ <DH/ BN/ JFR>L/'],
                  '3ms':['NPC'], '>JC':['SLX[']}

hyponyms_Lev19 = {'L H <NJ/ W L H GR/': ['GR#2'], 'MWT[':['>JC','CPXH']}

In [ ]:
show('JD<NJ',Lev19.AllRefs)

In [ ]:
tree = Lev19.tree(humanRefs_Lev19, synonyms_Lev19, hyponyms_Lev19)

In [ ]:
Lev19.treeStructure()
createPNG(tree, 'Leviticus', 19)

In [ ]:
Lev19.resultRefs()

In [ ]:
error_list_Lev19 = Lev19.checkResults()
print(len(error_list_Lev19))

## Lev 20

In [ ]:
Lev20 = Nodes.GenerateNodes('Leviticus',20)
Lev20.allRefs()
Lev20.AllRefs

* NDH: Refers to a woman who is menstruating
* 3mp#3: Like other similar references, refer to two or more participants grouped together. Part-whole relationships
* KL#2: Refers to creeping animals so is not included
* '>T NPC/ -BN JFR>L': Refers to the soul of the Israelites. The reference is included because "soul" is deeply personal.
* Is GWJ "nations" and <M "people" synonymous? >M seems to refer to a broader category of which GWJ is one particular people.
* '>JC >CH "a man or a woman": Generic like >JC
* FRP refers to a third person masculine verb (probably synonoymous with <M ARY

In [ ]:
humanRefs_Lev20 = ['JHWH','MCH=','BN JFR>L','>JC','GR','ZR< ->JC','MLK=','<M >RY','KL','NPC','>WB JD<NJ','>B ->JC >M ->JC',
                  '>B ->JC','>M ->JC','N>P N>P','CNJM -','CNJM -#2','>CH','CNJM -#3','>CH >M ->CH',
                  '2ms','>CH BHMH','BT >B ->JC BT >M ->JC','3unknownp','>CH#2','CNJM -#4','3ms','3mp','3mp#2','>X ->JC',
                  'NDH','3mp#3','GWJ','<M','>JC >CH','MN BN/ JFR>L/ W MN H GR/','MN QRB/ <M/ ->JC','>T CM/ QDC/ -JHWH',
                   'MN QRB/ <M/ -KL','MN QRB/ <M/ -NPC',
                   'B H >JC/ H HW> W B MCPXT/ ->JC','W >T KL/',
                   '>XR/ ->JC','>XR/ ->WB JD<NJ','>T >CH/ >JC/','>T >CH/ R<=/ ->JC','>T== >CH/ >B/ ->JC','<RWH/ >B/ ->JC',
                   '>T== KLH/ ->JC','>T== ZKR=/','FRP[','W','>T <RWH/ -BT >B ->JC BT >M ->JC','>T <RWH/ ->JC',
                   '<RWH/ >XWT/ ->JC','>T <RWH/ ->CH','MN QRB/ <M/ -CNJM -','<RWH/ >XWT/ >M/ -2msW >XWT/ >B/ -2ms',
                   '<RWH/ DWD/ ->JC','<RWH/ >X/ ->JC','>T== DWDH/ ->JC','>T >CH/ >X/ ->JC','MN PNH/ -BN JFR>L',
                   '>T NPC/ -BN JFR>L']

land_references_Lev20 = ['JFR>L','>RY','>DMH -GWJ','>RY/ ZWB[ XLB/ W DBC/']

In [ ]:
synonyms_Lev20 = {'JHWH':['>T CM/ QDC/ -JHWH'],'>JC': ['NPC','MN BN/ JFR>L/ W MN H GR/','>XR/ ->JC','>T <RWH/ ->JC', '3ms'],
                  'BN JFR>L':['MN PNH/ -BN JFR>L','>T NPC/ -BN JFR>L'],'<M >RY':['FRP['],
                  'MN QRB/ <M/ ->JC':['MN QRB/ <M/ -NPC'],
                  '>T== >CH/ >B/ ->JC':['<RWH/ >B/ ->JC'],
                  'BT >B ->JC BT >M ->JC':['<RWH/ >XWT/ ->JC','>T <RWH/ -BT >B ->JC BT >M ->JC'],
                  '>T >CH/ >X/ ->JC':['NDH'],'>WB JD<NJ':['>XR/ ->WB JD<NJ'],'>T >CH/ >JC/':['>T >CH/ R<=/ ->JC'],
                 '>CH#2': ['>T <RWH/ ->CH'], '>T== DWDH/ ->JC': ['<RWH/ DWD/ ->JC'], '>T >CH/ >X/ ->JC': ['<RWH/ >X/ ->JC']}

hyponyms_Lev20 = {'W >T KL/':['>JC','KL'],'>B ->JC >M ->JC':['>B ->JC','>M ->JC'],'N>P N>P':['>JC','>T >CH/ >JC/'],
                  'CNJM -':['>JC','>T== >CH/ >B/ ->JC'],'CNJM -#2':['>JC','>T== KLH/ ->JC'],'CNJM -#3':['>JC','>T== ZKR=/'],
                  'CNJM -#4':['>JC','>CH#2'],'3unknownp':['>JC','BT >B ->JC BT >M ->JC'],
                  '3mp':['>JC', '<RWH/ >XWT/ >M/ -2msW >XWT/ >B/ -2ms'],
                  '3mp#2':['>JC','>T== DWDH/ ->JC'],'>CH BHMH':['>CH'],'3mp#3':['>JC','>T >CH/ >X/ ->JC'],'<M':['GWJ'],
                  'B H >JC/ H HW> W B MCPXT/ ->JC':['>JC'],'W':['>JC','>CH >M ->CH']}

In [ ]:
show('MN QRB/ <M/ -CNJM -',Lev20.AllRefs,944353)

In [ ]:
tree = Lev20.tree(humanRefs_Lev20, synonyms_Lev20, hyponyms_Lev20)
Lev20.treeStructure()
createPNG(tree, 'Leviticus', 20)

In [ ]:
Lev20.resultRefs()

In [ ]:
error_list_Lev20 = Lev20.checkResults()
print(len(error_list_Lev20))

## Lev 21

In [ ]:
Lev21 = Nodes.GenerateNodes('Leviticus',21)
Lev21.allRefs()
Lev21.AllRefs

* 'L NPC/' refers to a dead body but may still be seen as a reference to a generic person.
* 'CM/ >LHJM/ -BN >HRN' refers to the name of God, essentially God himself.
* 'C>R -HW>' "his closest relatives" is technically a part of <M -HW> "his kinsmen", but in the text the two entites are rather contrasted rather than derived. Therefore not hyponyms.
* '>T >B/ -BT >JC KHN' "The father of any priest's daugther" seems to be synonymous to "priest", that is, 'BN >HRN'
* BTWLH/ is synonymous with '>CH/' "wife". '>CH/', however, is not really a participant but a title.
* '>JC#2' is specific man who is either blind, lame or mutilated, synonymous to 'KL/ >JC/'

In [ ]:
humanRefs_Lev21 = ['JHWH','MCH=','BN >HRN','HW>','<M -HW>','C>R -HW>','BTWLH','>LHJM -BN >HRN','>CH','2ms','BT >JC KHN',
                  'KHN','>LHJM -KHN','>CH#2','>LMNH GRC XLL','<M -KHN','>HRN','>JC','DWR ->JC','>LHJM ->JC','>JC#2','L NPC/',
                  'L >M/ -HW>W L >B/ -HW>W L BN/ -HW>W L BT/ -HW>W L >X/ -HW>','CM/ >LHJM/ -BN >HRN','>CH/ ZNH[ W XLL/',
                  'MN >JC/ ->CH','L >LHJM/ -HW>','2mp_sfx','>T >B/ -BT >JC KHN','MN >X/ -KHN','BTWLH/','<L KL/ NPC/ MWT[',
                   'L >B/ -KHN W L >M/ -KHN','ZR</ -KHN',
                   'KL/ >JC/','KL/ >JC/#2','>L >HRN/ W >L BN/ ->HRN W >L KL/ BN/ JFR>L/']

land_references_Lev21 = []

In [ ]:
synonyms_Lev21 = {'JHWH':['>LHJM -BN >HRN','>LHJM -KHN','>LHJM ->JC','CM/ >LHJM/ -BN >HRN','L >LHJM/ -HW>'],
                 'BN >HRN':['HW>', '>T >B/ -BT >JC KHN'],'BTWLH/': ['>CH#2'],'>LMNH GRC XLL': ['>CH'],
                  'C>R -HW>':['L >M/ -HW>W L >B/ -HW>W L BN/ -HW>W L BT/ -HW>W L >X/ -HW>','BTWLH'],
                 '>JC':['DWR ->JC','KL/ >JC/#2'], 'KL/ >JC/':['>JC#2'], 'L NPC/': ['<L KL/ NPC/ MWT[']}

hyponyms_Lev21 = {'>L >HRN/ W >L BN/ ->HRN W >L KL/ BN/ JFR>L/':['>HRN','BN >HRN']}

In [ ]:
show('KL/ >JC/',Lev21.AllRefs)

In [ ]:
tree = Lev21.tree(humanRefs_Lev21, synonyms_Lev21, hyponyms_Lev21)

In [ ]:
createPNG(tree, 'Leviticus', 21)

In [ ]:
Lev21.resultRefs()

In [ ]:
error_list_Lev21 = Lev21.checkResults()
len(error_list_Lev21)

## Lev 22

In [ ]:
Lev22 = Nodes.GenerateNodes('Leviticus',22)
Lev22.allRefs()
Lev22.AllRefs

In [ ]:
checkList(Lev22.AllRefs)

* 'CM QDC -JHWH': Refers to the name of YHWH and therefore to YHWH himself.
* '>JC' refers to the offspring of both Aaron and his sons (synonymous to NPC)
* '>JC >JC' refers to the offspring of Aaron alone (synonymous to NPC#2)
* '>JC#2' is a lay person
* '>DM' is generic (cf. v. 5)
* '>JC >JC#2' is generic, either Israelite of sojourner.

In [ ]:
humanRefs_Lev22 = ['JHWH','MCH=','>HRN BN ->HRN','>HRN','BN ->HRN','BN JFR>L','CM QDC -JHWH','>JC','NPC','>JC >JC','>JC#2',
                   '>DM','NPC#2','NPC#3','MN L PNH/ -JHWH',
                  'ZR','TWCB KHN FKJR','KHN','JLJD BJT -KHN','BT KHN','>B -BT KHN','>HRN BN ->HRN KL BN JFR>L','>JC >JC#2',
                  'L DWR/ ->HRN BN ->HRN','MN ZR</ >HRN/','B KL/ VM>/ NPC/','KL/ ZR/','ZR</','KL/ ZR/#2','B TWK/ BN/ JFR>L/']

land_references_Lev22 = ['JFR>L']

In [ ]:
synonyms_Lev22 = {'JHWH':['CM QDC -JHWH','MN L PNH/ -JHWH'],'>JC':['NPC'],
                  'MN ZR</ >HRN/':['>JC >JC','KHN','>B -BT KHN','NPC#2'],
                 'ZR':['KL/ ZR/','KL/ ZR/#2','>JC#2'], 'BN JFR>L':['B TWK/ BN/ JFR>L/']}

hyponyms_Lev22 = {'>HRN BN ->HRN KL BN JFR>L': ['>HRN BN ->HRN','BN JFR>L'], '>HRN BN ->HRN':['>HRN','BN ->HRN']}

In [ ]:
show('>DM',Lev22.AllRefs, 944526)

In [ ]:
tree = Lev22.tree(humanRefs_Lev22, synonyms_Lev22, hyponyms_Lev22)
createPNG(tree, 'Leviticus', 22)

In [ ]:
Lev22.resultRefs()

In [ ]:
error_list_Lev22 = Lev22.checkResults()
print(len(error_list_Lev22))

## Lev 23

In [ ]:
Lev23 = Nodes.GenerateNodes('Leviticus',23)
Lev23.allRefs()
Lev23.AllRefs

* 'NPC -BN JFR>L' refers to the soul of the Israelites, therefore (almost) synonymous to BN JFR>L.
* 'NPC' is generic. Seems to exemplify prohibitions related to BN JFR>L. '>ZRX' is also generic so is treated as a synonym to NPC.

In [ ]:
humanRefs_Lev23 = ['JHWH','MCH=','BN JFR>L','KHN','PNH JHWH','>LHJM -BN JFR>L','DWR -BN JFR>L','2ms','NPC -BN JFR>L','NPC',
                  '>ZRX','L H <NJ/ W L H GR/','MN <M/ -NPC','MN QRB/ <M/ -NPC','KL/ H >ZRX/']

land_references_Lev23 = ['JFR>L','>RY','MN >RY/ MYRJM/']

In [ ]:
synonyms_Lev23 = {'JHWH':['PNH JHWH','>LHJM -BN JFR>L'], 'BN JFR>L': ['NPC -BN JFR>L'], '>ZRX':['KL/ H >ZRX/'],
                 'MN <M/ -NPC':['MN QRB/ <M/ -NPC']}

hyponyms_Lev23 = {}

In [ ]:
#show('MN <M/ -NPC',Lev23.AllRefs)

In [ ]:
tree = Lev23.tree(humanRefs_Lev23, synonyms_Lev23, hyponyms_Lev23)
createPNG(tree, 'Leviticus', 23)

In [ ]:
Lev23.resultRefs()

In [ ]:
error_list_Lev23 = Lev23.checkResults()
print(len(error_list_Lev23))

## Lev 24

In [ ]:
Lev24 = Nodes.GenerateNodes('Leviticus',24)
Lev24.allRefs()
Lev24.AllRefs

* '3mp' refers to a particular group of witnesses (cf. v. 14)
* '<DH' "congregation" likely refers to BN JFR>L (cf. v. 23)
* Both >JC and >JC >JC are generic and therefore synonymous
* '>DM' is also generic but is distinct from >JC and >JC >JC.
* '>KL[ -QDC QDC' refers to Aaron and his sons (v. 9)
* '<L R>C/ -QLL' "on the head of the curser" refers to the curser.

In [ ]:
humanRefs_Lev24 = ['JHWH','MCH=','BN JFR>L','>HRN','PNH JHWH','>HRN BN ->HRN','BN >CH','>CH','BN JFR>LJ >JC JFR>LJ',
                   'CM JHWH','3mp','CLMJT','QLL','CM<','<DH','>JC >JC','GR','>JC','>DM','L DWR/ -BN JFR>L','KL/ H CM<[',
                  '<L R>C/ -QLL','KL/ H <DH/','>LHJM/ ->JC >JC','KL/ H <DH/#2','KL/ NPC/ >DM/','B <MJT/ ->JC']

land_references_Lev24 = ['JFR>L']

In [ ]:
synonyms_Lev24 = {'JHWH':['PNH JHWH','CM JHWH','>LHJM/ ->JC >JC'], '3mp':['CM<','KL/ H CM<['], 'CLMJT':['>CH'],
                  'BN >CH':['QLL','<L R>C/ -QLL'], 'BN JFR>L':['<DH','KL/ H <DH/','KL/ H <DH/#2'], '>JC >JC':['>JC'],
                 '>DM': ['KL/ NPC/ >DM/']}

hyponyms_Lev24 = {'BN JFR>LJ >JC JFR>LJ':['BN >CH'], '>HRN BN ->HRN':['>HRN']}

In [ ]:
#show('>HRN BN ->HRN',Lev24.AllRefs)

In [ ]:
tree = Lev24.tree(humanRefs_Lev24, synonyms_Lev24, hyponyms_Lev24)
createPNG(tree, 'Leviticus', 24)

In [ ]:
Lev24.resultRefs()

In [ ]:
error_list_Lev24 = Lev24.checkResults()
print(len(error_list_Lev24))

## Lev 25

In [ ]:
Lev25 = Nodes.GenerateNodes('Leviticus',25)
Lev25.allRefs()
Lev25.AllRefs

* GR TWCB refers to BN JFR>L
* '>JC' is generic. Sometimes synonymous to "your brother"
* TWCB specifies GR, therefore synonymous.
* Are >X -2ms and <MJT -2ms synonymous? Probably (cf. v. 14)
* GR TWCB#2 refers to the status of "your brother", therefore synonymous.
* '<BD -JHWH' refers to the BN JFR>L (= GR TWCB)
* 'QNH ->X -2ms' refers to the stranger (GR#2)
* 'L KL/ JCB[ ->RY' "all inhabitants of the land": Probably refers to both the Israelites and the so-journers.
* '3ms' seems to be "one of the Levites" (v. 33)

In [ ]:
humanRefs_Lev25 = ['JHWH','MCH=','GR TWCB','2ms','>MH -2ms','>JC','<MJT -2ms','>LHJM -2ms','GR','TWCB','>X -2ms',
                  'G>L ->X -2ms','G>L','JD ->JC','>JC#2','QNH','LWJ','3ms','JD ->X -2ms','HW>BN ->X -2ms',
                   'BN ->X -2ms','<BD -JHWH','<BD -2ms>MH -2ms','SBJB -GR TWCB','BN TWCB','BN -GR TWCB','>X -GR TWCB',
                   'JD GR TWCB','GR#2','>XD','DWD ->X -2msBN DWD ->X -2ms','DWD ->X -2ms','QNH ->X -2ms',
                  'W L <BD/ -2msW L >MH/ -2msW L FKJR/ -2msW L TWCB/ -2ms','GWR[','L KL/ JCB[ ->RY','>L MCPXT/ ->JC',
                  'MN JD/ <MJT/ -2ms','>T >X/ ->JC','>T <MJT/ ->JC','<MD/ -JHWH','B TWK/ BN/ JFR>L/',
                   'MN','MN#2','>L MCPXT/ ->X -2ms','MN MCPXT/ -BN TWCB','>XR/ -GR TWCB','L <QR=/ MCPXT/ GR/']

land_references_Lev25 = ['>RY','>RY MYRJM','B KL/ >RY/ -GR TWCB','>T >RY/ KN<N/']

In [ ]:
synonyms_Lev25 = {'JHWH':['>LHJM -2ms','<MD/ -JHWH'],'2ms':['MN#2'],'>JC':['JD ->JC'],
                  '>X -2ms': ['<MJT -2ms','MN JD/ <MJT/ -2ms','MN','JD ->X -2ms'], 'LWJ':['3ms'],
                  'GR TWCB':['GR','TWCB','<BD -JHWH','>XR/ -GR TWCB','B TWK/ BN/ JFR>L/'],
                  '>T >X/ ->JC':['>T <MJT/ ->JC'],'GR#2':['JD GR TWCB','QNH ->X -2ms']}

hyponyms_Lev25 = {'HW>BN ->X -2ms':['>X -2ms','BN ->X -2ms'],
                  'W L <BD/ -2msW L >MH/ -2msW L FKJR/ -2msW L TWCB/ -2ms':['2ms','<BD -2ms>MH -2ms','GWR['],
                  '<BD -2ms>MH -2ms':['>MH -2ms'], 'DWD ->X -2msBN DWD ->X -2ms':['DWD ->X -2ms']}

In [ ]:
show('GR TWCB',Lev25.AllRefs)

In [ ]:
tree = Lev25.tree(humanRefs_Lev25, synonyms_Lev25, hyponyms_Lev25)
createPNG(tree, 'Leviticus', 25)

In [ ]:
Lev25.resultRefs()

In [ ]:
error_list_Lev25 = Lev25.checkResults()
print(len(error_list_Lev25))

## Lev 26

In [ ]:
Lev26 = Nodes.GenerateNodes('Leviticus',26)
Lev26.allRefs()
Lev26.AllRefs

* '<M' refers to BN JFR>L in other chapters
* GWJ seems to be synonymous with the enemies of the people.
* 'HM' refers to a remnant of the Israelites.
* '>JC' is generic - a generic person within the group of HM (the remnant)

In [ ]:
humanRefs_Lev26 = ['<M','JHWH','>JB -<M','M>H','NPC -JHWH','MYRJM','NPC -<M','L PNH/ >JB[ -<M','FN> -<M','RDP',
                  '<FR= >CH','GWJ','HM','LBB -HM','>JB -HM','>JC','>B -HM','NPC -HM','L PNH/ -<M','XMC','M>H/','RBBH/',
                  'L PNH/ -<M#2','B TWK/ -<M','B TWK/ -<M#2','<BD/','<JN/','NPC/','PNH/ -JHWH','B TWK/ -<M#3','B JD/ >JB[',
                  'BFR/ BN/ -<M','BFR/ BT/ -<M','>XR/ -<M','B >X/ ->JC','J<QB/','JYXQ/','>BRHM/',
                  'BJN/ -JHWH W BJN/ BN/ JFR>L/','>T PGR/ -<M']

land_references_Lev26 = ['>RY -<M','>RY','>RY MYRJM','>RY >JB -HM','>RY >JB -<M']

In [ ]:
synonyms_Lev26 = {'<M':['M>H','NPC -<M','L PNH/ -<M','XMC','L PNH/ -<M#2','B TWK/ -<M','B TWK/ -<M#2','<BD/','<JN/','NPC/',
                       'B TWK/ -<M#3','>XR/ -<M'],
                  'JHWH':['NPC -JHWH','PNH/ -JHWH'],
                  '>JB -<M':['L PNH/ >JB[ -<M','FN> -<M','GWJ','M>H/','RBBH/','B JD/ >JB['],
                 'HM':['LBB -HM','NPC -HM']}

hyponyms_Lev26 = {'BJN/ -JHWH W BJN/ BN/ JFR>L/':['JHWH','<M']}

In [ ]:
show('L PNH/ >JB[ -<M',Lev26.AllRefs)

In [ ]:
tree = Lev26.tree(humanRefs_Lev26, synonyms_Lev26, hyponyms_Lev26)
createPNG(tree, 'Leviticus', 26)

In [ ]:
Lev26.resultRefs()

In [ ]:
error_list_Lev26 = Lev26.checkResults()
print(len(error_list_Lev26))

## Export nodes

In [ ]:
nodes_to_export = [Lev17.resultRefs(), Lev18.resultRefs(), Lev19.resultRefs(), Lev20.resultRefs(), Lev21.resultRefs(),
                   Lev22.resultRefs(), Lev23.resultRefs(), Lev24.resultRefs(), Lev25.resultRefs(), Lev26.resultRefs()]

In [ ]:
file_name = 'human_references_for_plotting.csv'
header = f'book,chapter,actor,references'

with open(file_name, 'w') as f:
    f.write('''{}'''.format(header))
    for chap in nodes_to_export:
        for actor in chap:
            bo, ch = T.sectionFromNode(chap[actor][0])[:2]
            references = ' '.join(str(e) for e in chap[actor])
            f.write('''\n{},{},{},{}'''.format(bo, ch, actor, references))